<a href="https://colab.research.google.com/github/singhsukhendra/MovieGenrePrediction/blob/master/High_performance_implementation_of_Extreme_Learning_Machines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

In [0]:
! git clone https://github.com/akusok/hpelm.git

In [0]:
! pip install hpelm

In [0]:
import hpelm

In [0]:
from sklearn.datasets import make_moons, make_regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [0]:
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
matplotlib.rcParams['figure.figsize'] = [10, 8]

In [0]:
X, Y = make_moons(n_samples=2000000, noise=0.3)
Xt, Xs, Yt, Ys = train_test_split(X, Y, test_size=0.3)
subs = 1000  # subsampling for plots
L = 100  # number of non-linear neurons

In [0]:
plt.scatter(Xs[Ys==0, 0][::subs], Xs[Ys==0, 1][::subs], c="r")
plt.scatter(Xs[Ys==1, 0][::subs], Xs[Ys==1, 1][::subs], c="b")
plt.show()

In [0]:
rfc = RandomForestClassifier(n_estimators=8, n_jobs=-1)
%time rfc.fit(Xt, Yt)
Yh = rfc.predict(Xs)

In [0]:
plt.scatter(Xs[Yh==0, 0][::subs], Xs[Yh==0, 1][::subs], c="r")
plt.scatter(Xs[Yh==1, 0][::subs], Xs[Yh==1, 1][::subs], c="b")
plt.show()

In [0]:
elm = hpelm.HPELM(2, 1, precision='double', norm=1)

In [0]:
elm.add_neurons(2, 'lin')
elm.add_neurons(L, 'tanh')

In [0]:
%time elm.train(Xt, Yt)

In [0]:
Yh = elm.predict(Xs)

In [0]:
Yh = np.array(Yh[:,0] > 0.5, dtype=np.int)

In [0]:
plt.scatter(Xs[Yh==0, 0][::subs], Xs[Yh==0, 1][::subs], c="r")
plt.scatter(Xs[Yh==1, 0][::subs], Xs[Yh==1, 1][::subs], c="b")
plt.show()

# Run HPELM+GPU

In [0]:
import os

In [0]:
elm2 = hpelm.HPELM(2, 1, precision='single', accelerator="GPU", norm=1)

In [0]:
!pip install scikit-cuda

In [0]:
elm2 = hpelm.HPELM(2, 1, precision='single', accelerator="GPU", norm=1)

In [0]:
elm2.add_neurons(2, 'lin')
elm2.add_neurons(L, 'tanh')

In [0]:
%time elm2.train(Xt, Yt)

In [0]:
Yh = elm2.predict(Xs)

In [0]:
Yh = np.array(Yh[:,0] > 0.5, dtype=np.int)

In [0]:
plt.scatter(Xs[Yh==0, 0][::subs], Xs[Yh==0, 1][::subs], c="r")
plt.scatter(Xs[Yh==1, 0][::subs], Xs[Yh==1, 1][::subs], c="b")
plt.show()

# Dummy task with very large **data**

In [0]:
X_big = np.random.randn(10000000, 200)  # 10 million samples

In [0]:
import numpy as np
# outputs are sums of first 10 and second 10 input values
Y_big = np.stack((X_big[:, :10].sum(1), X_big[:, 10:20].sum(1))).T
Y_big.shape

In [0]:
elm_cpu = hpelm.HPELM(200, 2)
elm_gpu = hpelm.HPELM(200, 2, precision="single", accelerator="GPU")

In [0]:
elm_cpu.add_neurons(200, 'lin')
elm_cpu.add_neurons(1000, 'tanh')

elm_gpu.add_neurons(200, 'lin')
elm_gpu.add_neurons(1000, 'tanh')

In [0]:
%%time
elm_cpu.train(X_big, Y_big)

In [0]:
%%time
elm_gpu.train(X_big, Y_big)

## plots checking that output weights correctly correspond to sum of first 10 inputs and the next 10 **inputs**

In [0]:
from matplotlib import pyplot as plt
B_cpu = elm_cpu.nnet.get_B()
plt.plot(range(1200), B_cpu[:,0])
plt.plot(range(1200), B_cpu[:,1])
plt.xlim([0, 50])
plt.show()

In [0]:
B_gpu = elm_gpu.nnet.get_B()  # this corectly downloads weights from GPU back to main memory
plt.plot(range(1200), B_gpu[:,0])
plt.plot(range(1200), B_gpu[:,1])
plt.xlim([0, 50])
plt.show()